In [ ]:
# bibliotecas
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import numpy as np
import torch

from tensorflow.keras.applications import Xception
from tensorflow.keras import layers, models

from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit

from torch.utils.data import Subset

### Carregamento e Pré-Processamento do Dataset

In [ ]:
from utils import data_utils

image_size = (299, 299)
train_data, test_data = data_utils.load_and_preprocess_data(resize_to=image_size,
                                                            keras_format=True,
                                                            model_type=data_utils.ModelType.XCEPTION)

### Divisão Holdout

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
train_idx, val_idx = next(split.split(np.zeros(len(train_data.targets)), train_data.targets))


In [ ]:
train_split = Subset(train_data, train_idx)
val_split = Subset(train_data, val_idx)

In [ ]:
print(f"Treino:    {len(train_split)} imagens")
print(f"Validação: {len(val_split)} imagens")
print(f"Teste:     {len(test_data)} imagens")

In [ ]:
def get_X_y(dataset):
    X = []
    y = []
    for img, label in dataset:
        X.append(img.numpy())  
        y.append(label)
    X = np.stack(X)
    y = np.array(y)
    return X, y


In [ ]:
X_train, y_train = get_X_y(train_split)
X_val, y_val = get_X_y(val_split)
X_test, y_test = get_X_y(test_data)

## Treino

### Modelo

**Xception**

A Xception Architecture é uma arquitetura de rede neural convolucional que foi introduzida em 2016 por François Chollet, o criador da biblioteca de deep learning Keras. Essa arquitetura é conhecida por sua eficiência e desempenho superior em comparação com outras arquiteturas convolucionais tradicionais. A Xception Architecture é baseada no conceito de depthwise separable convolutions, que consiste em dividir a convolução em duas etapas separadas: uma etapa para lidar com a convolução espacial e outra para lidar com a convolução em profundidade. Isso permite que a rede aprenda representações mais ricas e complexas, resultando em um melhor desempenho em tarefas de visão computacional.

Diferentemente de arquiteturas como a Inception, que combinam múltiplos tipos de convolução em paralelo, a Xception simplifica esse processo ao aplicar uma convolução depthwise (que opera separadamente em cada canal de entrada) seguida por uma convolução pointwise (1x1), que combina os canais. Essa separação permite um modelo mais leve e profundo, com menor número de parâmetros e operações, sem comprometer a capacidade de aprendizado. A arquitetura é composta por três partes principais: uma entrada com camadas convolucionais iniciais, um corpo formado por 14 módulos principais baseados em depthwise separable convolutions com conexões residuais, e uma saída com pooling global e uma camada densa para classificação.

Fonte: https://iatracker.com.br/glossario/o-que-e-xception-architecture/

In [ ]:
base_model = Xception(weights='imagenet', include_top=False, input_shape=(image_size, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32
)


In [ ]:
y_pred_prob = model.predict(X_test).ravel()
y_pred = (y_pred_prob > 0.5).astype(int)

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Real', 'Fake'], yticklabels=['Real', 'Fake'])
plt.xlabel("Predito")
plt.ylabel("Real")
plt.title("Matriz de Confusão")
plt.show()

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=["Real", "Fake"]))

roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f"AUC (ROC): {roc_auc:.4f}")

plt.figure(figsize=(8, 4))
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title("Curva de Aprendizado - Acurácia")
plt.xlabel("Época")
plt.ylabel("Acurácia")
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(8, 4))
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title("Curva de Aprendizado - Perda")
plt.xlabel("Época")
plt.ylabel("Perda (Loss)")
plt.legend()
plt.grid()
plt.show()
